In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!pip install cellpose

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.0/205.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 11.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinu

In [14]:
# I/O #
INPUT = "/content/drive/MyDrive/KI 67/DEV/Datasets/EZE004" #imagenes de entrada
IMAGE_FORMATS = ["jpg"]
OTUPUT_IA = '/content/drive/MyDrive/KI 67/DEV/Datasets/EZE004/IA' # almacenamiento de la mascara
OTUPUT_MASK = '/content/drive/MyDrive/KI 67/DEV/Datasets/EZE004/Mask' # almacenamiento de la mascara

FILE_MODEL_JSON = '/content/drive/MyDrive/KI 67/DEV/Modelos/Simple_2Clases/netk2_simple.json'
FILE_WEIGHTS = '/content/drive/MyDrive/KI 67/DEV/Modelos/Simple_2Clases/netk2_simple.weights.h5'

IMG_HEIGHT = 48
IMG_WIDTH = 48
IMG_CHANNELS = 3
METRICS = ['accuracy']
LOSS = 'categorical_crossentropy'
from tensorflow.keras.optimizers import SGD

OPT = SGD(learning_rate = 0.01)

# CELLPOSE #
MODEL_TYPE = "cyto2"
SEGMENT_CHANNEL = 2
DIAMETER = None
FLOW_THRESHOLD = 0.6
CELLPROB_THRESHOLD = 0

In [15]:
import tensorflow as tf
import numpy as np
from keras.backend import set_learning_phase
from os import walk,makedirs
from os.path import join, exists, basename, splitext
from cellpose.io import imread,imsave
from cv2 import normalize,NORM_MINMAX,CV_8U
from cellpose.models import Cellpose
from skimage.color import label2rgb
from keras.optimizers import SGD
from skimage.measure import regionprops
from skimage.transform import resize

# CLASIFICACION #
DIM_MAX = 25

# CARGAR LAS IMAGENES
files = [join(root, file) for root, _, files in walk(INPUT)
         for file in files if any(file.endswith(ext) for ext in IMAGE_FORMATS)]

# PARA GUARDAR LAS IMAGENES
if not exists(OTUPUT_IA):
    makedirs(OTUPUT_IA)
if not exists(OTUPUT_MASK):
    makedirs(OTUPUT_MASK)

# PREPARAR LOS MODELOS
model = Cellpose(model_type=MODEL_TYPE)
json_file = open(FILE_MODEL_JSON, 'r')

netk2 = json_file.read()
json_file.close()
set_learning_phase(1)
loaded_model = tf.keras.models.model_from_json(netk2)
loaded_model.load_weights(FILE_WEIGHTS)
loaded_model.compile(loss=LOSS, optimizer=OPT, metrics=METRICS)

for img_file in files:
    im = imread(img_file)
    n_dim = len(im.shape)
    dim = im.shape
    channel = np.min(dim)
    channel_position = dim.index(channel)
    if n_dim == 3 and channel_position == 0:
        im = im.transpose(1, 2, 0)
        dim = im.shape

    mask, _, _, _ = model.eval(im, diameter=DIAMETER, flow_threshold=FLOW_THRESHOLD,
                                cellprob_threshold=CELLPROB_THRESHOLD, channels=[SEGMENT_CHANNEL, 1])
    mask = mask.astype(np.uint16)

    total_celulas = np.max(mask)

100%|██████████| 25.3M/25.3M [00:00<00:00, 39.9MB/s]
100%|██████████| 3.54k/3.54k [00:00<00:00, 2.65MB/s]
/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


In [17]:
print(total_celulas)

145
